# StyleGAN2 전이학습을 이용한 학습방법과 이미지 생성 

**Content**

*   데이터셋 모으기(image scraping)
*   얼굴 크롭&리사이즈
*   styleGAN 전이학습
*   이미지 생성




데이터는 구글, 네이버, 핀터레스트 등 이미지 스크레이핑함

구글 비전 api로 얼굴인식 후 크롭

스타일갠에서 학습할 수 있게 (256x256 or 512x512 or 1024x1024)로 리사이징까지 완료 

이미지 파일 zip으로 묶어서 구글 드라이브에 준비

# styleGAN 전이학습




In [1]:
# GPU환경에서만 작동, GPU사용 확인 
!nvidia-smi

Thu Nov  4 15:09:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
# 텐서플로  1.x 에서만 동작하므로 버전 1로 연결 
%tensorflow_version 1.x

TensorFlow 1.x selected.



**StyleGAN2-ADA repo 복사 및 학습할 이미지 불러오기 위해 구글드라이브 연결**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# StyleGAN2-ADA repo 있으면 불러오고 없으면 설치 
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada"):
    %cd "/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada"
else:
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada
    %cd colab-sg2-ada
    
    #fork of the official repo. 
    !git clone https://github.com/dvschultz/stylegan2-ada
    %cd stylegan2-ada
    !mkdir downloads
    !mkdir datasets

/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada


In [ ]:
# 이미지 집파일 압축 풀기 
zip_path = "/content/drive/MyDrive/gan/crop_resize.zip"
!unzip {zip_path} -d /content/

Archive:  /content/drive/MyDrive/gan/crop_resize.zip
   creating: /content/crop_resize/
  inflating: /content/crop_resize/resized_crop_face36.jpg  
  inflating: /content/crop_resize/resized_crop_face22.jpg  
  inflating: /content/crop_resize/resized_crop_face134.jpg  
  inflating: /content/crop_resize/resized_crop_face120.jpg  
  inflating: /content/crop_resize/resized_crop_face108.jpg  
  inflating: /content/crop_resize/resized_crop_face109.jpg  
  inflating: /content/crop_resize/resized_crop_face121.jpg  
  inflating: /content/crop_resize/resized_crop_face135.jpg  
  inflating: /content/crop_resize/crop_face8_resized.jpg  
  inflating: /content/crop_resize/resized_crop_face23.jpg  
  inflating: /content/crop_resize/resized_crop_face37.jpg  
  inflating: /content/crop_resize/resized_crop_face21.jpg  
  inflating: /content/crop_resize/resized_crop_face35.jpg  
  inflating: /content/crop_resize/resized_crop_face123.jpg  
  inflating: /content/crop_resize/resized_crop_face137.jpg  
  inf

styleGAN에서 학습데이터는 tfrecord 포맷을 요구함 

이미지 데이터를 .tfrecords 파일 포맷으로 변경


In [ ]:
# 변경할 데이터셋이 들어있는 경로 
dataset_path = "/content/crop_resize"

# 변경 후 저장할 폴더이름 
dataset_name = "tfrecord_img"

!python dataset_tool.py create_from_images ./datasets/{dataset_name} {dataset_path}

Loading images from "/content/crop_resize"
Creating dataset "./datasets/tfrecord_img"
dataset_tool.py:97: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  'data': tf.train.Feature(bytes_list=tf.train.BytesList(value=[quant.tostring()]))}))
Added 234 images.


In [17]:
# 변경 후 저장할 폴더이름 
dataset_name = "tfrecord_img"

**전이학습에 필요한 기본옵션 지정**

In [16]:
# train에 가능한 옵션들 볼수 있는 명령어 
!python train.py --help

usage: train.py [-h] --outdir DIR [--gpus INT] [--snap INT] [--seed INT] [-n]
                --data PATH [--res INT] [--mirror BOOL] [--mirrory BOOL]
                [--use-raw BOOL] [--metrics LIST] [--metricdata PATH]
                [--cfg {auto,11gb-gpu,11gb-gpu-complex,24gb-gpu,24gb-gpu-complex,48gb-gpu,48gb-2gpu,stylegan2,paper256,paper512,paper1024,cifar,cifarbaseline,aydao}]
                [--lrate FLOAT] [--ttur BOOL] [--gamma FLOAT] [--nkimg INT]
                [--kimg INT] [--topk FLOAT] [--aug {noaug,ada,fixed,adarv}]
                [--p FLOAT] [--target TARGET] [--initstrength INITSTRENGTH]
                [--augpipe {blit,geom,color,filter,noise,cutout,bg,bgc,bgcf,bgcfn,bgcfnc}]
                [--cmethod {nocmethod,bcr,zcr,pagan,wgangp,auxrot,spectralnorm,shallowmap,adropout}]
                [--dcap FLOAT] [--resume RESUME] [--freezed INT]

Train a GAN using the techniques described in the paper
"Training Generative Adversarial Networks with Limited Data".

optional

In [14]:
# 모델이 샘플과 .pkl 을 생성하는 빈도 
snapshot_count = 4

# x축 미러링
mirrored = True

# y축 미러링
mirroredY = False

# 코랩은 gpu사용한계가 있기 때문에 메트릭 없이 하는것을 추천
metric_list = None

# augments
augs = "bgc"

# 첫 전이학습은 "ffhq512" 로 지정 
# 전이학습후 저장된 마지막 피클파일 경로로 지정하면 이어서 학습 가능 
resume_from = "ffhq512"


In [ ]:
# 옵션지정 후 학습시작 명령어 
!python train.py --outdir ./results --snap={snapshot_count} --cfg=11gb-gpu --data=./datasets/{dataset_name} --augpipe={augs} --mirror={mirrored} --mirrory={mirroredY} --metrics={metric_list} --resume={resume_from} --augpipe="bg"


tcmalloc: large alloc 4294967296 bytes == 0x558a73428000 @  0x7f6400f22001 0x7f63fe12554f 0x7f63fe175b58 0x7f63fe179b17 0x7f63fe218203 0x558a6cdbb544 0x558a6cdbb240 0x558a6ce2f627 0x558a6ce29ced 0x558a6cdbd48c 0x558a6cdfe159 0x558a6cdfb0a4 0x558a6cdbbd49 0x558a6ce2f94f 0x558a6ce299ee 0x558a6ccfbe2b 0x558a6ce2bfe4 0x558a6ce299ee 0x558a6ccfbe2b 0x558a6ce2bfe4 0x558a6ce29ced 0x558a6ccfbe2b 0x558a6ce2bfe4 0x558a6cdbcafa 0x558a6ce2a915 0x558a6ce299ee 0x558a6ce296f3 0x558a6cef34c2 0x558a6cef383d 0x558a6cef36e6 0x558a6cecb163
tcmalloc: large alloc 4294967296 bytes == 0x558b73428000 @  0x7f6400f201e7 0x7f63fe12546e 0x7f63fe175c7b 0x7f63fe17635f 0x7f63fe218103 0x558a6cdbb544 0x558a6cdbb240 0x558a6ce2f627 0x558a6ce299ee 0x558a6cdbcbda 0x558a6ce2b737 0x558a6ce299ee 0x558a6cdbcbda 0x558a6ce2b737 0x558a6ce299ee 0x558a6cdbcbda 0x558a6ce2b737 0x558a6cdbcafa 0x558a6ce2a915 0x558a6ce299ee 0x558a6cdbcbda 0x558a6ce2ed00 0x558a6ce299ee 0x558a6cdbcbda 0x558a6ce2b737 0x558a6ce29ced 0x558a6cdbd48c 0x558a6cdf

모델이 학습하면서 체크포인트마다 .pkl 파일과 .jpg 파일 출력

jpg파일은 지금까지 학습한 모델을 사용해 변환시킨 이미지를 보여줌 

# 이미지 생성 

In [ ]:
# 이미지 생성에 필요한 라이브러리 설치 
!pip install opensimplex


**generate.py** : 시드별 이미지 생성

seeds : 저장된 시드별 오리지널이미지에 학습한 모델의 스타일을 입힐수 있음 


(10, 20 단일지정 or 0-30 처럼 범위지정)

network : 학습한 모델(.pkl file)경로

시드와 모델 지정후 이미지 generate > out 폴더에 저장 

In [ ]:
!python generate.py generate-images --trunc=1 --seeds=85,265,297,849 --network=/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00000-tf_img-mirror-11gb-gpu-bg-resumeffhq512/network-snapshot-000080.pkl

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)360448/45929032 bytes (0.8%)2932736/45929032 bytes (6.4%)6987776/45929032 bytes (15.2%)11182080/45929032 bytes (24.3%)15335424/45929032 bytes (33.4%)19415040/45929032 bytes (42.3%)23470080/45929032 bytes (51.1%)27451392/45929032 bytes (59.8%)31399936/45929032 bytes (68.4%)35266560/45929032 bytes (76.8%)39321600/45929032 bytes (85.6%)43376640/45929032 bytes (94.4%)459

**projector.py** : 내가 원하는 이미지에 styleGAN입힌 이미지 생성 

outdir : 이미지 저장될 위치 

target : 변경할 이미지 경로

save_video : 변환과정 비디오 저장 여부 


실행하면 변환된 이미지와 dlatents.npz 파일 저장 




In [ ]:
!python projector.py --outdir=out --target=/content/drive/MyDrive/test_img/resized_crop_face9.jpg \
    --save-video=False --network=/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00001-tfrecord_img-mirror-11gb-gpu-bg-resumeffhq512/network-snapshot-000080.pkl


dlatents.npz파일을 사용해 generate.py 에서 dlatents 경로지정하면 같은 이미지 재생성 가능 

In [ ]:
!python generate.py --outdir=out --dlatents=out/dlatents.npz \ # dlatents 지정
    --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl

**style_mixing.py** : 좀 더 세부적인 스타일 변경

rows, cols seed를 지정해서 스타일 믹싱 

In [ ]:
!python style_mixing.py --outdir=out --rows=1 --cols=2 \
--network=/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results/00001-tfrecord_img-mirror-11gb-gpu-bg-resumeffhq512/network-snapshot-000080.pkl